In [58]:
#Required general libraries are imported
import os
import csv
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import string
import nltk
import re
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_curve, accuracy_score, recall_score, classification_report, f1_score, average_precision_score, precision_recall_fscore_support, roc_auc_score
from catboost import CatBoostClassifier
import xgboost as XGB
from imblearn.over_sampling import SMOTE



In [59]:
#importing data into jupyter
comment = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
comment

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [60]:
# Setting new variables labels 
comment = comment.rename(columns={'Review Text': 'Review_Text', 'Recommended IND': 'Recommended_IND', 'Class Name': 'Class_Name'})
comment = comment[['Title', 'Review_Text', 'Rating', 'Class_Name','Recommended_IND']]

In [61]:
comment['Text_Length'] = comment['Review_Text'].str.len()

In [62]:
rate = px.histogram(comment, y="Rating", orientation='h', 
                    color_discrete_sequence=['olive'],
                    title=f" Comments  Length to Rating")
rate.update_yaxes(type='category')
rate.show()

In [63]:
hist_comp = px.histogram(comment, y="Class_Name", orientation='h', 
                    color_discrete_sequence=['green'],
                    title=f" Product purchase per Class")
hist_comp.show()

In [64]:
Hist_class = px.histogram(comment, 
            x = "Class_Name", 
            y = "Recommended_IND", 
            color = "Recommended_IND", 
            barmode = "group",
            histnorm = 'percent',
            histfunc = "count",
            color_discrete_sequence = ["orange", "gray"], 
            title = "Class Name recommendation per class ")

Hist_class.update_layout(yaxis={"ticksuffix":"%"})
Hist_class.update_layout(yaxis_title="Percentage")
Hist_class.update_xaxes(type='category')
Hist_class.show()

In [65]:
comment["Text"] = comment["Review_Text"].str.lower()

In [66]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [67]:
#Remove punctuation

def remove_punctuation(punc):
    rem_punc = [i for i in punc if i not in string.punctuation]
    after_punc = ''.join(rem_punc)
    return after_punc

In [68]:
#remove regular expressions

# write a regular expression to identify urls in text
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# write a regular expression to identify non-ascii characters in text
non_ascii_regex = r'[^\x00-\x7F]+'

In [69]:
## write a function to regular expression cleaning

def clean_re(text_variable):
    
    # use library re to replace urls by token - urlplaceholder
    text_variable = re.sub(url_regex, 'urlplaceholder', text_variable)    
    
    # use library re to replace non ascii characters by a space
    text_variable = re.sub(non_ascii_regex, ' ', text_variable)

    return text_variable 

In [75]:
# LEMMATIZATION

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [76]:
def clean_re(text_variable):
    
    # use library re to replace urls by token - urlplaceholder
    text_variable = re.sub(url_regex, 'urlplaceholder', text_variable)    
    
    # use library re to replace non ascii characters by a space
    text_variable = re.sub(non_ascii_regex, ' ', text_variable)

    return text_variable 

In [80]:
pos_df = comment[comment.Recommended_IND == 1]
neg_df = comment[comment.Recommended_IND == 0]

In [81]:
#common words analysis

def top_n_ngram(corpus, n = None, ngram = 1):
    vec = CountVectorizer(stop_words = 'english',ngram_range=(ngram,ngram)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis =0) 
    words_freq = [(word,sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq,key = lambda x:x[1],reverse = True)
    return words_freq[:n]